In [1]:
import numpy as np
import scipy.io as sio
import json
import csv
import pandas as pd

In [2]:
filename = 'species_list.csv'

df_species = pd.read_csv(filename, index_col=0)
    
print(df_species.shape)

(10721, 9)


In [3]:
df_species.head(10)

,order,family_name,family_description,family_code,code,com_name,sci_name,genus,species
0,Struthioniformes,Struthionidae,Ostriches,struth1,ostric2,Common Ostrich,Struthio camelus,Struthio,camelus
1,Struthioniformes,Struthionidae,Ostriches,struth1,ostric3,Somali Ostrich,Struthio molybdophanes,Struthio,molybdophanes
2,Rheiformes,Rheidae,Rheas,rheida1,grerhe1,Greater Rhea,Rhea americana,Rhea,americana
3,Rheiformes,Rheidae,Rheas,rheida1,lesrhe2,Lesser Rhea,Rhea pennata,Rhea,pennata
4,Tinamiformes,Tinamidae,Tinamous,tinami1,tabtin1,Tawny-breasted Tinamou,Nothocercus julius,Nothocercus,julius
5,Tinamiformes,Tinamidae,Tinamous,tinami1,higtin1,Highland Tinamou,Nothocercus bonapartei,Nothocercus,bonapartei
6,Tinamiformes,Tinamidae,Tinamous,tinami1,hootin1,Hooded Tinamou,Nothocercus nigrocapillus,Nothocercus,nigrocapillus
7,Tinamiformes,Tinamidae,Tinamous,tinami1,grytin1,Gray Tinamou,Tinamus tao,Tinamus,tao
8,Tinamiformes,Tinamidae,Tinamous,tinami1,soltin1,Solitary Tinamou,Tinamus solitarius,Tinamus,solitarius
9,Tinamiformes,Tinamidae,Tinamous,tinami1,blatin1,Black Tinamou,Tinamus osgoodi,Tinamus,osgoodi


## iNaturalist 2017 classes

In [4]:
import json

DATAPATH = '/home/pf/pfstaff/projects/andresro/infusion/data/iNaturalist'

In [5]:
train_file = DATAPATH+'/2017/train_val2017/train2017.json'

with open(train_file) as json_file:
    train_annotations = json.load(json_file)

val_file = DATAPATH+'/2017/train_val2017/val2017.json'

with open(val_file) as json_file:
    val_annotations = json.load(json_file)

test_file = DATAPATH+'/2017/test2017/test2017.json'

with open(test_file) as json_file:
    test_annotations = json.load(json_file)


In [6]:
train_annotations.keys()

dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])

In [7]:
names_inat = [x['name'] for x in train_annotations['categories'] if x['supercategory'] == 'Aves']

df1 = df_species[df_species.sci_name.isin(names_inat)]
print(df1.shape)
sci_names_inat = list(df1.sci_name)


(895, 9)


In [8]:
train_subset = [x for x  in  train_annotations['images'] if x['file_name'].split('/')[2] in sci_names_inat]

val_subset = [x for x  in  val_annotations['images'] if x['file_name'].split('/')[2] in sci_names_inat]


In [9]:
print('Train  complete {} subset {}'.format(len(train_annotations['images']), len(train_subset)))

print('Val  complete {} subset {}'.format(len(val_annotations['images']), len(val_subset)))

print('Test  complete {} subset'.format(len(test_annotations['images'])))

Train  complete 579184 subset 202943
Val  complete 95986 subset 19699
Test  complete 182707 subset


In [10]:
train_keys = [x['file_name'][:-4] for x in train_subset]

### Create class splits

In [146]:
np.random.seed(1)

results = []
for order,  group in df1.groupby('order'):
    for family, group1 in group.groupby('family_name'):
        for genus, group2 in group1.groupby('genus'):
            for species, group3 in group2.groupby('sci_name'):
                results.append({'order':order,'n_families':group['family_name'].drop_duplicates().shape[0],
                                'family':family,'n_genus':group1['genus'].drop_duplicates().shape[0],
                                'genus':genus,'n_species':group2['sci_name'].drop_duplicates().shape[0],
                                'sci_name':species,'is_train':np.random.random() > 0.2
                                })

In [147]:
df2 = pd.DataFrame(results)
print(df2.shape)
df2.head(10)

(895, 8)


,order,n_families,family,n_genus,genus,n_species,sci_name,is_train
0,Accipitriformes,3,Accipitridae,19,Accipiter,4,Accipiter cooperii,True
1,Accipitriformes,3,Accipitridae,19,Accipiter,4,Accipiter gentilis,True
2,Accipitriformes,3,Accipitridae,19,Accipiter,4,Accipiter nisus,False
3,Accipitriformes,3,Accipitridae,19,Accipiter,4,Accipiter striatus,True
4,Accipitriformes,3,Accipitridae,19,Aquila,1,Aquila chrysaetos,False
5,Accipitriformes,3,Accipitridae,19,Busarellus,1,Busarellus nigricollis,False
6,Accipitriformes,3,Accipitridae,19,Buteo,10,Buteo albonotatus,False
7,Accipitriformes,3,Accipitridae,19,Buteo,10,Buteo brachyurus,True
8,Accipitriformes,3,Accipitridae,19,Buteo,10,Buteo buteo,True
9,Accipitriformes,3,Accipitridae,19,Buteo,10,Buteo jamaicensis,True


In [148]:
index = (df2['n_families'] > 1) & (df2['n_genus'] > 1) & (df2['n_species'] > 1)

df2.loc[~index, 'is_train'] = False
print(df2.is_train.mean())

df3 = df2[index]

0.42569832402234636


In [149]:
print(df3.shape)

# df2.head(10)

(477, 8)


In [150]:
df_train = df3[df3['is_train']]
df_train.shape

(381, 8)

In [151]:
df_val = df2[~df2['is_train']]
df_val.shape

(514, 8)

In [152]:
index_1hop = df_val.genus.isin(df_train.genus)

df_val_1hop = df_val[index_1hop]

df_val_1hop.shape

(87, 8)

In [153]:
index_2hop = df_val.family.isin(df_train.family) & ~df_val.genus.isin(df_train.genus)

df_val_2hop = df_val[index_2hop]

df_val_2hop.shape

(177, 8)

In [154]:
index_3hop = df_val.order.isin(df_train.order) & ~df_val.family.isin(df_train.family)

df_val_3hop = df_val[index_3hop]

df_val_3hop.shape

(110, 8)

In [155]:
index_4hop = ~df_val.order.isin(df_train.order)

df_val_4hop = df_val[index_4hop]

df_val_4hop.shape

(140, 8)

In [156]:
val_sets = {'train':df_train,
            'val_seen':df_train,
            'val allhop':df_val,
            'val 1hop':df_val_1hop,
            'val 2hop':df_val_2hop,
            'val 3hop':df_val_3hop,
            'val 4hop':df_val_4hop,}

In [11]:
is_overwrite = False 
import pickle

file = DATAPATH+'/2017/zsl_splits/all_splits.pickle'
if is_overwrite:
    with open(file, 'wb') as handle:
        pickle.dump(val_sets, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(file, 'rb') as handle:
        val_sets = pickle.load(handle)




In [13]:

df_train = val_sets['train']

list_out = []
for key, val in val_sets.items():
    
    list_ = list(val['sci_name'])
    
    dset_zsl = [x for x  in  train_subset if x['file_name'].split('/')[2] in list_]
    n1 = len(dset_zsl)
    # else:
    dset_zsl = [x for x  in  val_subset if x['file_name'].split('/')[2] in list_]
    n2 = len(dset_zsl)
    n3 = n1 if key == 'train' else n2
        
    # print( f' {key} n sci_name = {val.shape[0]} , n_images = {n1 + n2}')

    list_out.append({'Set':key,
        #  'n1':n1,'n2':n2,'n1+n2':n1+n2,
        'n_samples':n3})
    for level in ['order','family','genus','sci_name']:
        index_ = val[level].drop_duplicates().isin(df_train[level])
        list_out[-1][level+'_train'] = index_.sum()
        list_out[-1][level] = index_.shape[0]
        # print(f'\t {level} in train {index_.sum()} ({index_.mean():.1%})')




In [14]:
df_summary_17 = pd.DataFrame(list_out).set_index('Set')


In [15]:

def use_f_2(x):
    return f'{x:,.0f}'

df_summary_17[['n_samples','sci_name']].to_latex(
        # 'tables/inat17_splits.tex',
        formatters=[use_f_2,use_f_2]
        )

'\\begin{tabular}{lrr}\n\\toprule\n{} & n\\_samples & sci\\_name \\\\\nSet        &           &          \\\\\n\\midrule\ntrain      &    97,067 &      381 \\\\\nval\\_seen   &     8,626 &      381 \\\\\nval allhop &    11,073 &      514 \\\\\nval 1hop   &     2,204 &       87 \\\\\nval 2hop   &     3,613 &      177 \\\\\nval 3hop   &     2,175 &      110 \\\\\nval 4hop   &     3,081 &      140 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [164]:
save_dir = DATAPATH+'/2017/zsl_splits/'

is_overwrite = False
if is_overwrite:
    df_train['sci_name'].to_csv(save_dir + "seen_classes.txt", index = False,header=False)
    df_val['sci_name'].to_csv(save_dir + "unseen_allhop_classes.txt", index = False,header=False)
    df_val_1hop['sci_name'].to_csv(save_dir + "unseen_1hop_classes.txt", index = False,header=False)
    df_val_2hop['sci_name'].to_csv(save_dir + "unseen_2hop_classes.txt", index = False,header=False)
    df_val_3hop['sci_name'].to_csv(save_dir + "unseen_3hop_classes.txt", index = False,header=False)
    df_val_4hop['sci_name'].to_csv(save_dir + "unseen_4hop_classes.txt", index = False,header=False)

    df2['sci_name'].to_csv(save_dir + "all_classes.txt", index = False,header=False)

## iNaturalist 2021 classes

In [4]:
import json

In [5]:
train_file = DATAPATH+'/2021/train.json'

with open(train_file) as json_file:
    train_annotations = json.load(json_file)

train_mini_file = DATAPATH+'/2021/train_mini.json'

with open(train_mini_file) as json_file:
    train_mini_annotations = json.load(json_file)

val_file = DATAPATH+'/2021/val.json'

with open(val_file) as json_file:
    val_annotations = json.load(json_file)


test_file = DATAPATH+'/2021/public_test.json'

with open(test_file) as json_file:
    test_annotations = json.load(json_file)

In [6]:
test_annotations.keys()

dict_keys(['info', 'images', 'categories', 'licenses'])

In [7]:
names_inat = [x['name'] for x in val_annotations['categories'] if x['supercategory'] == 'Birds']

In [8]:
dirs_val = [x['image_dir_name'] for x in val_annotations['categories'] if x['supercategory'] == 'Birds']

In [21]:
len(names_inat)

1486

In [22]:
df_species.columns

Index(['order', 'family_name', 'family_description', 'family_code', 'code',
       'com_name', 'sci_name', 'genus', 'species'],
      dtype='object')

In [9]:
# subseting to only species that have samples in Billow
df_illustrations = pd.read_csv('/home/pf/pfstaff/projects/andresro/birds/dataset/illustrations_list.txt', index_col=0)

df_merged = df_illustrations.reset_index().set_index('sci_name').join(df_species.set_index('sci_name'), how='left').reset_index().set_index('index').sort_index()
# 
df_count = df_merged.groupby('sci_name')['sample'].count()
df_count = df_count[df_count > 0]
df2 = df_count[df_count.index.isin(names_inat)]
print(df2.shape)

sci_names_inat = list(df2.index)

df1 = df_species[df_species.sci_name.isin(sci_names_inat)]
print(df1.shape)


(1485,)
(1485, 9)


In [10]:
get_sciname = lambda x: ' '.join(x.split('/')[1].split('_')[-2:])    

In [11]:
train_subset = [x for x  in  train_annotations['images'] if get_sciname(x['file_name']) in sci_names_inat]

train_mini_subset = [x for x  in  train_mini_annotations['images'] if get_sciname(x['file_name']) in sci_names_inat]

val_subset = [x for x  in  val_annotations['images'] if get_sciname(x['file_name']) in sci_names_inat]

In [12]:
print('Train  complete {} subset {}'.format(len(train_annotations['images']), len(train_subset)))
print('Train-mini  complete {} subset {}'.format(len(train_mini_annotations['images']), len(train_mini_subset)))
print('Val  complete {} subset {}'.format(len(val_annotations['images']), len(val_subset)))
print('Test  complete {}'.format(len(test_annotations['images'])))

Train  complete 2686843 subset 414547
Train-mini  complete 500000 subset 74250
Val  complete 100000 subset 14850
Test  complete 500000


### Create class splits

In [27]:
df1

,order,family_name,family_description,family_code,code,com_name,sci_name,genus,species
0,Struthioniformes,Struthionidae,Ostriches,struth1,ostric2,Common Ostrich,Struthio camelus,Struthio,camelus
53,Casuariiformes,Casuariidae,Cassowaries and Emu,casuar1,emu1,Emu,Dromaius novaehollandiae,Dromaius,novaehollandiae
60,Anseriformes,Anhimidae,Screamers,anhimi1,souscr1,Southern Screamer,Chauna torquata,Chauna,torquata
62,Anseriformes,Anseranatidae,Magpie Goose,ansera1,maggoo1,Magpie Goose,Anseranas semipalmata,Anseranas,semipalmata
65,Anseriformes,Anatidae,"Ducks, Geese, and Waterfowl",anatid1,wfwduc1,White-faced Whistling-Duck,Dendrocygna viduata,Dendrocygna,viduata
...,...,...,...,...,...,...,...,...,...
10683,Passeriformes,Thraupidae,Tanagers and Allies,thraup2,bkfgra,Black-faced Grassquit,Melanospiza bicolor,Melanospiza,bicolor
10707,Passeriformes,Thraupidae,Tanagers and Allies,thraup2,butsal1,Buff-throated Saltator,Saltator maximus,Saltator,maximus
10709,Passeriformes,Thraupidae,Tanagers and Allies,thraup2,blhsal1,Black-headed Saltator,Saltator atriceps,Saltator,atriceps
10711,Passeriformes,Thraupidae,Tanagers and Allies,thraup2,grasal1,Grayish Saltator,Saltator coerulescens,Saltator,coerulescens


In [28]:
np.random.seed(1)

results = []
for order,  group in df1.groupby('order'):
    for family, group1 in group.groupby('family_name'):
        for genus, group2 in group1.groupby('genus'):
            for species, group3 in group2.groupby('sci_name'):
                results.append({'order':order,'n_families':group['family_name'].drop_duplicates().shape[0],
                                'family':family,'n_genus':group1['genus'].drop_duplicates().shape[0],
                                'genus':genus,'n_species':group2['sci_name'].drop_duplicates().shape[0],
                                'sci_name':species,'is_train':np.random.random() > 0.2
                                })

In [29]:
df2 = pd.DataFrame(results)
print(df2.shape)
df2.head(10)

(1485, 8)


,order,n_families,family,n_genus,genus,n_species,sci_name,is_train
0,Accipitriformes,3,Accipitridae,34,Accipiter,6,Accipiter badius,True
1,Accipitriformes,3,Accipitridae,34,Accipiter,6,Accipiter cooperii,True
2,Accipitriformes,3,Accipitridae,34,Accipiter,6,Accipiter gentilis,False
3,Accipitriformes,3,Accipitridae,34,Accipiter,6,Accipiter nisus,True
4,Accipitriformes,3,Accipitridae,34,Accipiter,6,Accipiter striatus,False
5,Accipitriformes,3,Accipitridae,34,Accipiter,6,Accipiter trivirgatus,False
6,Accipitriformes,3,Accipitridae,34,Aegypius,1,Aegypius monachus,False
7,Accipitriformes,3,Accipitridae,34,Aquila,5,Aquila audax,True
8,Accipitriformes,3,Accipitridae,34,Aquila,5,Aquila chrysaetos,True
9,Accipitriformes,3,Accipitridae,34,Aquila,5,Aquila heliaca,True


In [30]:
index = (df2['n_families'] > 1) & (df2['n_genus'] > 1) & (df2['n_species'] > 1)

df2.loc[~index, 'is_train'] = False
print(df2.is_train.mean())

df3 = df2[index]

0.5043771043771044


In [31]:
print(df3.shape)

# df2.head(10)

(927, 8)


In [32]:
df_train = df3[df3['is_train']]
df_train.shape

(749, 8)

In [33]:
df_val = df2[~df2['is_train']]
df_val.shape

(736, 8)

In [34]:
index_1hop = df_val.genus.isin(df_train.genus)

df_val_1hop = df_val[index_1hop]

df_val_1hop.shape

(168, 8)

In [35]:
index_2hop = df_val.family.isin(df_train.family) & ~df_val.genus.isin(df_train.genus)

df_val_2hop = df_val[index_2hop]

df_val_2hop.shape

(286, 8)

In [36]:
index_3hop = df_val.order.isin(df_train.order) & ~df_val.family.isin(df_train.family)

df_val_3hop = df_val[index_3hop]

df_val_3hop.shape

(158, 8)

In [37]:
index_4hop = ~df_val.order.isin(df_train.order)

df_val_4hop = df_val[index_4hop]

df_val_4hop.shape

(124, 8)

In [39]:
val_sets = {'train':df_train,
            'val_seen':df_train,
            'val allhop':df_val,
            'val 1hop':df_val_1hop,
            'val 2hop':df_val_2hop,
            'val 3hop':df_val_3hop,
            'val 4hop':df_val_4hop,}


is_overwrite = False
import pickle

file = DATAPATH+'/2021/zsl_splits/all_splits.pickle'
if is_overwrite:
    with open(file, 'wb') as handle:
        pickle.dump(val_sets, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(file, 'rb') as handle:
        val_sets = pickle.load(handle)


In [40]:


df_train = val_sets['train']

list_out = []
for key, val in val_sets.items():
    
    list_ = list(val['sci_name'])
    
    dset_zsl = [x for x  in  train_subset if get_sciname(x['file_name']) in list_]
    n1 = len(dset_zsl)
    # else:
    dset_zsl = [x for x  in  val_subset if get_sciname(x['file_name']) in list_]
    n2 = len(dset_zsl)
    n3 = n1 if key == 'train' else n2
        
    # print( f' {key} n sci_name = {val.shape[0]} , n_images = {n1 + n2}')

    list_out.append({'Set':key,
        #  'n1':n1,'n2':n2,'n1+n2':n1+n2,
        'n_samples':n3})
    for level in ['order','family','genus','sci_name']:
        index_ = val[level].drop_duplicates().isin(df_train[level])
        list_out[-1][level+'_train'] = index_.sum()
        list_out[-1][level] = index_.shape[0]
        # print(f'\t {level} in train {index_.sum()} ({index_.mean():.1%})')




In [41]:
df_summary_21 = pd.DataFrame(list_out).set_index('Set')
def use_f_2(x):
    return f'{x:,.0f}'

# df.to_latex(formatters=[None, use_f_2, use_f_2])

df_summary_21[['n_samples','sci_name']].to_latex(
        # 'tables/inat21_splits.tex',
        # float_format="{:0.2f}".format
        formatters=[use_f_2,use_f_2]
        )

'\\begin{tabular}{lrr}\n\\toprule\n{} & n\\_samples & sci\\_name \\\\\nSet        &           &          \\\\\n\\midrule\ntrain      &   211,027 &      749 \\\\\nval\\_seen   &     7,490 &      749 \\\\\nval allhop &     7,360 &      736 \\\\\nval 1hop   &     1,680 &      168 \\\\\nval 2hop   &     2,860 &      286 \\\\\nval 3hop   &     1,580 &      158 \\\\\nval 4hop   &     1,240 &      124 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [204]:
save_dir = DATAPATH+'/2021/zsl_splits/'

is_overwrite = False
if is_overwrite:
    df_train['sci_name'].to_csv(save_dir + "seen_classes.txt", index = False,header=False)
    df_val['sci_name'].to_csv(save_dir + "unseen_allhop_classes.txt", index = False,header=False)
    df_val_1hop['sci_name'].to_csv(save_dir + "unseen_1hop_classes.txt", index = False,header=False)
    df_val_2hop['sci_name'].to_csv(save_dir + "unseen_2hop_classes.txt", index = False,header=False)
    df_val_3hop['sci_name'].to_csv(save_dir + "unseen_3hop_classes.txt", index = False,header=False)
    df_val_4hop['sci_name'].to_csv(save_dir + "unseen_4hop_classes.txt", index = False,header=False)
    df2['sci_name'].to_csv(save_dir + "all_classes.txt", index = False,header=False)

In [42]:
df_summary_17

,n_samples,order_train,order,family_train,family,genus_train,genus,sci_name_train,sci_name
Set,,,,,,,,,
train,97067,13,13,44,44,132,132,381,381
val_seen,8626,13,13,44,44,132,132,381,381
val allhop,11073,13,30,43,114,60,363,0,514
val 1hop,2204,11,11,30,30,60,60,0,87
val 2hop,3613,13,13,40,40,0,172,0,177
val 3hop,2175,12,12,0,53,0,64,0,110
val 4hop,3081,0,17,0,18,0,67,0,140


In [43]:
df_summary_17['Year'] = 2017
df_summary_21['Year'] = 2021

In [68]:
df_summary = pd.concat((df_summary_17,df_summary_21))

In [69]:
df_summary.index, df_summary.columns

(Index(['train', 'val_seen', 'val allhop', 'val 1hop', 'val 2hop', 'val 3hop',
        'val 4hop', 'train', 'val_seen', 'val allhop', 'val 1hop', 'val 2hop',
        'val 3hop', 'val 4hop'],
       dtype='object', name='Set'),
 Index(['n_samples', 'order_train', 'order', 'family_train', 'family',
        'genus_train', 'genus', 'sci_name_train', 'sci_name', 'Year'],
       dtype='object'))

In [71]:
df_out = df_summary.reset_index().set_index(['Set','Year'])[['n_samples','sci_name']].unstack(1)[[('n_samples', 2017),
            ( 'sci_name', 2017),
            ('n_samples', 2021),
            ( 'sci_name', 2021)]].swaplevel(axis=1)
df_out.stack().T

Set      train           val 1hop           val 2hop           val 3hop  \
     n_samples sci_name n_samples sci_name n_samples sci_name n_samples   
Year                                                                      
2017     97067      381      2204       87      3613      177      2175   
2021    211027      749      1680      168      2860      286      1580   

Set            val 4hop          val allhop           val_seen           
     sci_name n_samples sci_name  n_samples sci_name n_samples sci_name  
Year                                                                     
2017      110      3081      140      11073      514      8626      381  
2021      158      1240      124       7360      736      7490      749

In [75]:
df_out.stack().T.to_latex(
        # 'tables/inat_splits.T.tex',
        # formatters=4*[use_f_2]
        )



'\\begin{tabular}{lrrrrrrrrrrrrrr}\n\\toprule\nSet & \\multicolumn{2}{l}{train} & \\multicolumn{2}{l}{val 1hop} & \\multicolumn{2}{l}{val 2hop} & \\multicolumn{2}{l}{val 3hop} & \\multicolumn{2}{l}{val 4hop} & \\multicolumn{2}{l}{val allhop} & \\multicolumn{2}{l}{val\\_seen} \\\\\n{} & n\\_samples & sci\\_name & n\\_samples & sci\\_name & n\\_samples & sci\\_name & n\\_samples & sci\\_name & n\\_samples & sci\\_name &  n\\_samples & sci\\_name & n\\_samples & sci\\_name \\\\\nYear &           &          &           &          &           &          &           &          &           &          &            &          &           &          \\\\\n\\midrule\n2017 &     97067 &      381 &      2204 &       87 &      3613 &      177 &      2175 &      110 &      3081 &      140 &      11073 &      514 &      8626 &      381 \\\\\n2021 &    211027 &      749 &      1680 &      168 &      2860 &      286 &      1580 &      158 &      1240 &      124 &       7360 &      736 &      7490 &     